In [1]:
!pip install -q gwpy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# !pip install rouge_score
# !pip install py7zr
# !pip install transformers
# !pip install datasets
# !pip install rouge
from typing import List
import os
import json
import ast
import pandas as pd
import csv
import argparse
import pickle
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
from datasets import load_dataset, load_metric, load_from_disk
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from rouge import Rouge
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nahyun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nahyun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\nahyun\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_parsed = pd.read_csv('./Results_csv/twcs_train.csv')

max_input_len = 0
max_target_len = 0

for i in range(len(train_parsed)):
    

In [2]:
max_input = 512
max_target = 128
batch_size = 100
# model_name= "philschmid/distilbart-cnn-12-6-samsum
model_name = "linydub/bart-large-samsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
def preprocess_data(data_to_process):
  #get all the dialogues  
  data_to_process['dialog'] = data_to_process['dialog'].astype('string')
  data_to_process['summary'] = data_to_process['summary'].astype('string')
  list_summary = list(data_to_process['summary'])
  inputs = [dialogue for dialogue in data_to_process['dialog']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(list_summary, max_length=max_target, padding='max_length', truncation=True)
  #set labels
  'id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'
  model_inputs['labels'] = targets['input_ids']

  #return the tokenized data
  #input_ids, attention_mask and labels
  results = []
  for i in range (len(list(data_to_process['dialog']))):
    dic = {'dialogue':list(data_to_process['dialog'])[i], 'summary':list(data_to_process['summary'])[i],'input_ids':model_inputs['input_ids'][i], 'attention_mask':model_inputs['attention_mask'][i], 'labels':model_inputs['labels'][i]}
    results.append(dic)
  return results



In [6]:
train_parsed = pd.read_csv('./Results_csv/twcs_train.csv')
valid_parsed = pd.read_csv('./Results_csv/twcs_val.csv')
test_parsed = pd.read_csv('./Results_csv/twcs_test.csv')

preprocessed_train_data = preprocess_data(train_parsed)
preprocessed_valid_data = preprocess_data(valid_parsed)

c:\Users\nahyun\anaconda3\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
# model_name="philschmid/distilbart-cnn-12-6-samsum"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
metric = load_metric('rouge')
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [14]:
args = Seq2SeqTrainingArguments(
    'conversation-summ', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=True
    )

trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=preprocessed_train_data,
    eval_dataset=preprocessed_valid_data,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [15]:
trainer.train()

c:\Users\nahyun\anaconda3\envs\torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2406
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 1803
  Number of trainable parameters = 406291456

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: dialogue, summary. If dialogue, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call

{'loss': 1.9338, 'learning_rate': 1.4498058790904051e-05, 'epoch': 0.83}


Model weights saved in conversation-summ\checkpoint-500\pytorch_model.bin
tokenizer config file saved in conversation-summ\checkpoint-500\tokenizer_config.json
Special tokens file saved in conversation-summ\checkpoint-500\special_tokens_map.json





































































































***** Running Evaluation *****
  Num examples = 327
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: dialogue, summary. If dialogue, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
















































































































































































































































































































{'eval_loss': 0.5691310167312622, 'eval_rouge1': 51.767, 'eval_rouge2': 28.3619, 'eval_rougeL': 45.0885, 'eval_rougeLsum': 45.2346, 'eval_gen_len': 46.0367, 'eval_runtime': 149.7194, 'eval_samples_per_second': 2.184, 'eval_steps_per_second': 1.095, 'epoch': 1.0}

















































































































































































































































































































































































































                                                                 

                                             
Downloading:   6%|▌         | 69.2M/1.22G [09:18<02:49, 6.81MB/s]
Saving model checkpoint to conversation-summ\checkpoint-1000
Configuration saved in conversation-summ\checkpoint-1000\config.json


...











































































































































































































































































































                                                                 

                                             
Downloading:   6%|▌         | 69.2M/1.22G [14:28<02:49, 6.81MB/s]
Saving model checkpoint to conversation-summ\checkpoint-1500
Configuration saved in conversation-summ\checkpoint-1500\config.json


{'loss': 0.3856, 'learning_rate': 3.4054353854686633e-06, 'epoch': 2.5}


Model weights saved in conversation-summ\checkpoint-1500\pytorch_model.bin
tokenizer config file saved in conversation-summ\checkpoint-1500\tokenizer_config.json
Special tokens file saved in conversation-summ\checkpoint-1500\special_tokens_map.json
Deleting older checkpoint [conversation-summ\checkpoint-500] due to args.save_total_limit















































































































































































































































































































***** Running Evaluation *****
  Num examples = 327
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: dialogue, summary. If dialogue, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.









{'eval_loss': 0.5461166501045227, 'eval_rouge1': 50.8752, 'eval_rouge2': 27.307, 'eval_rougeL': 44.1152, 'eval_rougeLsum': 44.2311, 'eval_gen_len': 47.2997, 'eval_runtime': 152.3205, 'eval_samples_per_second': 2.147, 'eval_steps_per_second': 1.077, 'epoch': 3.0}
{'train_runtime': 1035.0585, 'train_samples_per_second': 6.974, 'train_steps_per_second': 1.742, 'train_loss': 0.833894548717632, 'epoch': 3.0}


TrainOutput(global_step=1803, training_loss=0.833894548717632, metrics={'train_runtime': 1035.0585, 'train_samples_per_second': 6.974, 'train_steps_per_second': 1.742, 'train_loss': 0.833894548717632, 'epoch': 3.0})

In [ ]:
predicted_sum = []
for test_dialog in test_parsed['dialog']:
  model_inputs = tokenizer(test_dialog,  max_length=max_input, padding='max_length', truncation=True)
  raw_pred, _, _ = trainer.predict([model_inputs])
  predicted_sum.append(tokenizer.decode(raw_pred[0]))

In [ ]:
cleaned_data = []
for summary in predicted_sum:
  summary= summary.replace('</s>', '')
  summary= summary.replace('<s>', '')
  summary= summary.replace('<pad>', '')
  cleaned_data.append(summary)

In [ ]:
test_parsed['finetuned_predicted'] = cleaned_data

In [ ]:
summaries = list(test_parsed['summary'])

rouge = Rouge()
rouge.get_scores(list(test_parsed['finetuned_predicted']), summaries, avg=True)

In [ ]:
test_parsed.to_csv("predicted_with_finetuned_model.csv")

In [ ]:
def generation_Rouge_Scores(X, Y):  
  rouge = Rouge()
  return rouge.get_scores(X, Y)

def get_rouge1_rouge2_scores_for_bxplot(test_parsed):
  Rouge_1_f1scores_list = []
  Rouge_2_f1scores_list = []
  Rouge_1_precision_list = []
  Rouge_2_precision_list = []
  Rouge_1_recall_list = []
  Rouge_2_recall_list = []
  for i in range(len(test_parsed)):
    X = test_parsed['summary'][i]
    Y = test_parsed['finetuned_predicted'][i]
    get_rouge_scores = generation_Rouge_Scores(X,Y)
    Rouge_1_f1scores = get_rouge_scores[0]['rouge-1']['f']
    Rouge_2_f1scores = get_rouge_scores[0]['rouge-2']['f']

    Rouge_1_f1scores_list.append(Rouge_1_f1scores)
    Rouge_2_f1scores_list.append(Rouge_2_f1scores)

    Rouge_1_precision = get_rouge_scores[0]['rouge-1']['p']
    Rouge_2_precision = get_rouge_scores[0]['rouge-2']['p']

    Rouge_1_precision_list.append(Rouge_1_precision)
    Rouge_2_precision_list.append(Rouge_2_precision)

    Rouge_1_recall = get_rouge_scores[0]['rouge-1']['r']
    Rouge_2_recall = get_rouge_scores[0]['rouge-2']['r']

    Rouge_1_recall_list.append(Rouge_1_recall)
    Rouge_2_recall_list.append(Rouge_2_recall)
  return Rouge_1_f1scores_list, Rouge_2_f1scores_list

In [ ]:
Rouge_1_f1scores_finetuned_CNN, Rouge_2_f1scores_finetuned_CNN = get_rouge1_rouge2_scores_for_bxplot(test_parsed)
print("Max for Rouge_1 f1 scores    : ", np.max(Rouge_1_f1scores_finetuned_CNN))
print("Min for Rouge_1 f1 scores    : ", np.min(Rouge_1_f1scores_finetuned_CNN))
print("Mean for Rouge_1 f1 scores   : ", np.mean(Rouge_1_f1scores_finetuned_CNN))
print("Median for Rouge_1 f1 scores : ", np.median(Rouge_1_f1scores_finetuned_CNN))

In [ ]:
sns.set(style="darkgrid")
plt.title("Rouge 1 f1 scores for cnn-12-6-samsum model")
sns.boxplot(x=Rouge_1_f1scores_finetuned_CNN)

In [ ]:
print("Max for Rouge_2 f1 scores    : ", np.max(Rouge_2_f1scores_finetuned_CNN))
print("Min for Rouge_2 f1 scores    : ", np.min(Rouge_2_f1scores_finetuned_CNN))
print("Mean for Rouge_2 f1 scores   : ", np.mean(Rouge_2_f1scores_finetuned_CNN))
print("Median for Rouge_2 f1 scores : ", np.median(Rouge_2_f1scores_finetuned_CNN))

In [ ]:
sns.set(style="darkgrid")
plt.title("Rouge 2 f1 scores for cnn-12-6-samsum model")
sns.boxplot(x=Rouge_2_f1scores_finetuned_CNN)